In [17]:
# Webスクレピングに最低限必要なライブラリをインポート
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [12]:
# アクセスしたいWebサイトのURLを指定
url = 'https://www.mlit.go.jp/kankocho/tokei_hakusyo/shukuhakutokei.html'

# WebサーバーにHTTPリクエストを送信
# レスポンスを変数に格納しておく
res = requests.get(url)

In [13]:
res.encoding = res.apparent_encoding  # レスポンスのエンコーディングを設定

print(f"レスポンス：{res}")  # レスポンスオブジェクトを表示
print(f"レスポンスの型：{type(res)}")  # レスポンスオブジェクトの型を表示
print(f"ステータスコード：{res.status_code}")  # ステータスコードを表示
print(f"ステータスメッセージ：{res.reason}")  # ステータスメッセージを表示

print(f"リクエスト：{res.request}")  # リクエストオブジェクトを表示
print(f"リクエストの型：{type(res.request)}")  # リクエストオブジェクトの型を表示
print(f"リクエストヘッダー：{res.request.headers}")  # リクエストヘッダーを表示
print(f"リクエストメソッド：{res.request.method}")  # リクエストメソッドを表示
print(f"リクエストURL：{res.request.url}")  # リクエストURLを表示

レスポンス：<Response [200]>
レスポンスの型：<class 'requests.models.Response'>
ステータスコード：200
ステータスメッセージ：OK
リクエスト：<PreparedRequest [GET]>
リクエストの型：<class 'requests.models.PreparedRequest'>
リクエストヘッダー：{'User-Agent': 'python-requests/2.32.5', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
リクエストメソッド：GET
リクエストURL：https://www.mlit.go.jp/kankocho/tokei_hakusyo/shukuhakutokei.html


In [14]:
# レスポンスの HTML から BeautifulSoup オブジェクトを作る
soup = BeautifulSoup(res.text, 'html.parser')

In [26]:
# 1. サーバー負荷に配慮して1秒待機
time.sleep(1)

# 2. すでに特定したExcelのURLからデータを読み込む
excel_url = "https://www.mlit.go.jp/kankocho/content/001750679.xlsx"
excel_file = pd.ExcelFile(excel_url)
target_sheet = [s for s in excel_file.sheet_names if '第2表' in s][0]
df_clean = pd.read_excel(excel_url, sheet_name=target_sheet, header=9)

# 3. 必要な列（県名、日本人、外国人）を抽出して掃除
df_final = df_clean.iloc[:, [1, 2, 3]].copy()
df_final.columns = ['都道府県', '日本人宿泊者数', '外国人宿泊者数']

# 数値変換とノイズ除去
df_final['日本人宿泊者数'] = pd.to_numeric(df_final['日本人宿泊者数'], errors='coerce')
df_final['外国人宿泊者数'] = pd.to_numeric(df_final['外国人宿泊者数'], errors='coerce')
df_final = df_final.dropna(subset=['日本人宿泊者数', '外国人宿泊者数']).head(47)

# 4. 都道府県名を正しく割り当てる
prefectures = [
    "北海道", "青森県", "岩手県", "宮城県", "秋田県", "山形県", "福島県", "茨城県", "栃木県", "群馬県",
    "埼玉県", "千葉県", "東京都", "神奈川県", "新潟県", "富山県", "石川県", "福井県", "山梨県", "長野県",
    "岐阜県", "静岡県", "愛知県", "三重県", "滋賀県", "京都府", "大阪府", "兵庫県", "奈良県", "和歌山県",
    "鳥取県", "島根県", "岡山県", "広島県", "山口県", "徳島県", "香川県", "愛媛県", "高知県", "福岡県",
    "佐賀県", "長崎県", "熊本県", "大分県", "宮崎県", "鹿児島県", "沖縄県"
]
df_final['都道府県'] = prefectures

# 5. 比率の計算
df_final['インバウンド比率(%)'] = (df_final['外国人宿泊者数'] / (df_final['日本人宿泊者数'] + df_final['外国人宿泊者数'])) * 100

# 6. 【ここが重要！】ランキングリストの作成
# 日本人客数が多いトップ5
popular_prefectures = df_final.sort_values('日本人宿泊者数', ascending=False)['都道府県'].head(5).tolist()

# インバウンド比率が高いトップ5
inbound_prefectures = df_final.sort_values('インバウンド比率(%)', ascending=False)['都道府県'].head(5).tolist()

# 7. 結果の表示
print("外国人観光客（インバウンド）比率が高い県トップ5：", inbound_prefectures)
print("日本人に根強く人気の県トップ5：", popular_prefectures)

# DataFrameにまとめて見やすく表示
df_rank = pd.DataFrame({
    '外国人観光客急増県(比率)トップ5': inbound_prefectures,
    '日本人人気県(実数)トップ5': popular_prefectures
})
print("\n--- 比較分析結果 ---")
print(df_rank)

外国人観光客（インバウンド）比率が高い県トップ5： ['和歌山県', '茨城県', '秋田県', '長野県', '鳥取県']
日本人に根強く人気の県トップ5： ['沖縄県', '群馬県', '三重県', '愛知県', '大分県']

--- 比較分析結果 ---
  外国人観光客急増県(比率)トップ5 日本人人気県(実数)トップ5
0              和歌山県            沖縄県
1               茨城県            群馬県
2               秋田県            三重県
3               長野県            愛知県
4               鳥取県            大分県
